## Using Gemma without RAG


### Step 1: Install libraries

In [1]:
!pip install -q langchain transformers torch
!pip install langchain-community --quiet
!pip install langchain_huggingface --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00


### Step 2: Import Packages

In [2]:
#from langchain_community.llms import HuggingFacePipeline
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEndpoint
from huggingface_hub import notebook_login
import torch
import os
from google.colab import userdata

In [3]:
# Set up the Hugging Face Hub API token and login to huggingface_hub
os.environ['HUGGINGFACE_HUB_TOKEN'] = userdata.get('HUGGINGFACE_HUB_TOKEN')
notebook_login()

In [6]:
# Configuration
MODEL_NAME = "google/gemma-3-1b-it"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32
)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [8]:
# Create text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7
)

Device set to use cuda:0


In [9]:
# Wrap in LangChain
llm = HuggingFacePipeline(pipeline=pipe)

In [10]:
user_input = input("\nYou: ")

# Format prompt for Gemma
prompt = f"<start_of_turn>user\n{user_input}<end_of_turn>\n<start_of_turn>model\n"

# Generate response
response = llm.invoke(prompt)

# Extract just the model's response
if isinstance(response, list):
    response = response[0]['generated_text']

# Clean up the output
response = response.split("<start_of_turn>model\n")[-1].strip()
print(f"\nAssistant: {response}")


You: How sex is determined?

Assistant: Okay, let's break down how sex is determined. It's a really fascinating and complex topic, and scientists have made incredible progress in understanding it. Here's a breakdown of the current understanding, categorized into different aspects:

**1. The Basics: Chromosomes and Sex Chromosomes**

* **Chromosomes:**  Sex is primarily determined by the number and type of chromosomes an individual has.  Humans typically have 23 pairs of chromosomes: 22 pairs of autosomes (non-sex chromosomes) and one pair of sex chromosomes.
* **Sex Chromosomes:** These are the key players. Females typically have two X chromosomes (XX), while males have one X and one Y chromosome (XY).
* **X and Y Chromosomes:** The Y chromosome carries the *SRY* gene, which is crucial for male development. It's often called the "male-determining region."  The presence of the SRY gene is the primary factor in determining male sex.

**2. The Role of Genes**

* **SRY Gene:** As mentione

## Using Gemma with RAG

### Step 0: Install additional packages

In [11]:
!pip install -q -U chromadb pypdf sentence-transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 112.6 MB/s

In [12]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [26]:
# ======================
# CONFIGURATION
# ======================
PDF_FOLDER_PATH = "./Book_Class_10_Ch8/"  # Folder containing your PDFs
CHROMA_DB_PATH = "/content/drive/MyDrive/"  # Where to store ChromaDB
EMBEDDING_MODEL = "sentence-transformers/all-mpnet-base-v2"

In [14]:
# Create directories if they don't exist
os.makedirs(CHROMA_DB_PATH, exist_ok=True)

### Step 1: Data Preparation

*   Read text from PDF documents
*   Divide PDF document into smaller chunks  

In [27]:
# ======================
# LOAD AND CHUNK PDFs
# ======================
def load_and_chunk_pdfs(pdf_folder):
    documents = []

    # Load each PDF in the folder
    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, pdf_file)
            print(f"Processing {pdf_file}...")

            # Load PDF
            loader = PyPDFLoader(pdf_path)
            print('loading done')
            pages = loader.load()
            print('pages loaded')

            # Split into chunks
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200,
                length_function=len
            )
            chunks = text_splitter.split_documents(pages)
            documents.extend(chunks)

    print(f"\nTotal documents: {len(documents)}")
    return documents

### Step 2: Create VectorStore

In [17]:
# ======================
# CREATE VECTORSTORE
# ======================
def create_vectorstore(documents):
    # Initialize embedding function
    embedding_function = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL,
        model_kwargs={"device": DEVICE},
    )

    # Create and persist ChromaDB
    vector_db = Chroma.from_documents(
        documents=documents,
        embedding=embedding_function
    )
    return vector_db


### Step 3: Setup Gemma Q&A Chain


*   Get access tokens from Hugging face
*   Initialize the tokenizer with the model.
*   Create a text generation pipeline.
*   Initialize the LLM with pipeline and model arguments
*   The final step is to generate the answers using both the vector store and the LLM.

It will generate embeddings to the input query or question retrieve the context from the vector store, and feed this to the LLM to generate the answers

In [18]:
# ======================
# SETUP GEMMA QA CHAIN
# ======================
def setup_qa_chain(vector_db):
    # Load Gemma model
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32,
    )

    # Create pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )

    # Wrap in LangChain
    llm = HuggingFacePipeline(pipeline=pipe)

    # Custom prompt for Gemma
    prompt_template = """<start_of_turn>user
    Use the following context to answer the question. Be concise and accurate.
    If you don't know, say you don't know. Don't make up answers.

    Context: {context}

    Question: {question}<end_of_turn>
    <start_of_turn>model
    """

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    # Create QA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_db.as_retriever(search_kwargs={"k": 3}),
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents=True
    )

    return qa_chain

### Step 4: Main execution

In [29]:
# ======================
# MAIN EXECUTION
# ======================

# Step 1: Process PDFs
print("Loading and chunking PDFs...")
documents = load_and_chunk_pdfs(PDF_FOLDER_PATH)

# Step 2: Create vector store
print("\nCreating vector store...")
vector_db = create_vectorstore(documents)

# Step 3: Setup QA system
print("\nSetting up QA system...")
qa_chain = setup_qa_chain(vector_db)


Loading and chunking PDFs...
Processing jesc108.pdf...
loading done
pages loaded

Total documents: 22

Creating vector store...


<ipython-input-17-f4fee62760cb>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


Setting up QA system...


Device set to use cuda:0


In [31]:
# question answering

question = input("\nYour question: ")
result = qa_chain.invoke({"query": question})
response = result["result"].split("<start_of_turn>model\n")[-1].strip()
print("\nAnswer:", response)

# print("\nAnswer:", result["result"])



# Show sources
print("\nSources:")
for i, doc in enumerate(result["source_documents"], 1):
    print(f"\nSource {i}:")
    print("Content:", doc.page_content[:200] + "...")
    print("From:", doc.metadata.get("source", "Unknown"))


Your question: How gender is determined?

Answer: According to the text, gender is determined by what a child inherits from their father.

Sources:

Source 1:
Content: from their mother regardless of whether they are boys or
girls. Thus, the sex of the children will be determined by
what they inherit from their father. A child who inherits
an X chromosome from her f...
From: ./Book_Class_10_Ch8/jesc108.pdf

Source 2:
Content: inheritance might work?
8.2.4 Sex Determination
We have discussed the idea that the two sexes participating in sexual
reproduction must be somewhat different from each other for a number
of reasons. H...
From: ./Book_Class_10_Ch8/jesc108.pdf

Source 3:
Content: But so far, we have assumed that similar gene sets ar
e
inherited from both parents. If that is the case, how can
genetic inheritance determine sex?
The explanation lies in the fact that all human
chr...
From: ./Book_Class_10_Ch8/jesc108.pdf
